In [92]:
import tensorflow as tf 
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data')

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [20]:
def generator(z, Output_dimension, n_units, reuse, alpha):
    with tf.variable_scope('generator', reuse=reuse):
        # creat a variable scope to save the variable under the 'generator' variable list  
        H_layer1=tf.layers.dense(z, n_units, activation=None,name='G_dense_1')
        # hidden layer: input is latent variable, number of units= 128
        H_layer1=tf.maximum(H_layer1, alpha*H_layer1)
        # this maximum combine with dense layer can realize the Leaky ReLU 
        # alpha is a parameter which is less than 1
        logits=tf.layers.dense(H_layer1, Output_dimension, activation=None,name='G_dense_2')
        # logits layer:Output dimension=724 
        Out=tf.nn.tanh(logits)
        # output is rescaled into between -1 to 1 
        return Out, logits

In [94]:
def discriminator(x, n_units, reuse, alpha):
    with tf.variable_scope('discriminaor',reuse=reuse):
        # creat a variable scope to save the variable under the 'discriminator' variable list
        H_layer1=tf.layers.dense(x,n_units,activation=None,name='D_dense_1')
        # Hidden layer: input is the images to be discriminated, number of units=128
        H_layer1=tf.maximum(H_layer1,alpha*H_layer1)
        # smae as generator: creat the operation as Leaky ReLU
        logits=tf.layers.dense(H_layer1,1,activation=None,name='D_dense_2')
        # logits layer: output dimension is 1
        Out=tf.nn.sigmoid(logits)
        # output value: when it is close to 1, it will be treated as the real image, vice versa.
        return Out, logits
    

In [128]:
tf.reset_default_graph()# reset the default graph 
z=tf.placeholder(tf.float32,(None,100),name='z')
# creat placeholder for input latent variable z, dimension=[batch size, 100]
real_image=tf.placeholder(tf.float32,(None,724),name='real_image')
# creat placeholder for the real trainging data, dimension= [batch size, 724] 

In [129]:
# initial input parameter set here
Output_dimension=724
n_units=128
alpha=0.01
smooth=0.1


In [130]:
# creat generator network 
g_out,g_logits=generator(z,Output_dimension,n_units,False,alpha)
# creat discriminator network
d_out_real,d_logits_real=discriminator(real_image,n_units,False,alpha)
d_out_fake,d_logits_fake=discriminator(g_out,n_units,True,alpha)

In [142]:
# loss funtion goes here 
# firstly label the real image from the training data by 1 and fake image from the generetor by 0
# In order toreduce overfitting, use the label smoothing method here: it prevents a network
#from assigning full probability to each training example and maintains a reasonable ratio between
# the logits of the incorrect classes. 
#And from the experiment result of this paper smooth=0.1 always perform well
real_label=tf.ones_like(d_logits_real)*(1-smooth)
fake_label=tf.zeros_like(d_logits_fake)
# discriminator loss funtion 
d_loss_real=tf.nn.sigmoid_cross_entropy_with_logits(labels=real_label,logits=d_logits_real)
d_loss_fake=tf.nn.sigmoid_cross_entropy_with_logits(labels=fake_label,logits=d_logits_fake)
# here uing the sigmoid_cross_entropy, causing the label is between 0 to 1.
# discriminator loss funtion is that training the discriminator to assign the right label to real
# image 1 and to fake image 0 as precisely as possible
d_loss=tf.reduce_mean(d_loss_real+d_loss_fake)# total loss is the summation of both two parts
g_loss=tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.ones_like(d_logits_fake),
                                                              logits=d_logits_fake))
# generator loss funtion is that training the generator to generator fake images that are more like 
# real ones, that is the labels are close to 1.

In [140]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    Z= np.random.uniform(-1, 1, size=(16, 100))
    result=sess.run([g_loss],feed_dict={z:Z})
    

[0.8247317]